In [ ]:
from google.colab import drive
drive.mount("/content/drive/", force_remount = True)

##Imports

In [ ]:
pip install pytorch.msssim

In [ ]:
pip install sewar 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sewar: filename=sewar-0.4.5-py3-none-any.whl size=10608 sha256=9afb348d0aee816fad78fad2270dd8fad9be8e7e1796560cf35cfb93ca05469e
  Stored in directory: /root/.cache/pip/wheels/7a/38/6b/d066cfcb2b1c1c9b059b9ce1bdc4803b469359214e8dad4e3e
Successfully built sewar


In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
from google.colab.patches import cv2_imshow
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import copy
from torch.nn.functional import relu, leaky_relu, sigmoid
# import torch.tensor
from torch import Tensor
from torch.nn import TransformerEncoder, TransformerEncoderLayer, Dropout, BatchNorm1d, Conv2d, ConvTranspose2d, MultiheadAttention, Softmax, Softmax2d, Container, Module, ModuleList
from typing import Optional, Any
import torch.cuda
from torch import Tensor
from torch.utils.data import Dataset, DataLoader
from torch.nn import Dropout, Linear, LayerNorm, LogSoftmax
from torch.nn.functional import softmax , log_softmax
from torch.nn.init import xavier_uniform_
import glob
import argparse
import random
import torch.utils.data as data
import sys
from collections import OrderedDict
import torch.utils.model_zoo as model_zoo
import math
import time
import matplotlib
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
# from option import args
from pytorch_msssim import ssim
from torch.optim import Adam, lr_scheduler
from torch.utils.tensorboard import SummaryWriter

##Unzipping

In [ ]:
!unzip "/content/drive/MyDrive/SAR/Raw Data/ROIs1158_spring.zip" -d "/content/drive/MyDrive/SAR/Raw Data"

##Preprocessing

In [ ]:
#for spring data
#three numbers are 92,95,98
main_summer_path = "/content/drive/MyDrive/SAR/Raw Data/rois1868_summer"
num_files = os.listdir(main_spring_path)
s1_list = []
s2_list = []
for i in num_files:
    if 's1' in i:
        s1_list.append(int(i[3:]))
    else:
        s2_list.append(int(i[3:]))

s1_list.sort()
s2_list.sort()

print(s1_list)
print(s2_list)

[0, 4, 10, 18, 19, 20, 24, 25, 32, 38, 39, 41, 45, 47, 49, 51, 53, 54, 56, 57, 58, 59, 61, 75, 77, 78, 80, 85, 86, 88, 90, 92, 95, 102, 106, 110, 113, 115, 119, 120, 121, 127, 128, 132, 134, 135, 136, 142, 148]
[0, 4, 10, 18, 19, 20, 24, 25, 32, 38, 39, 41, 45, 47, 49, 51, 53, 54, 56, 57, 58, 59, 61, 75, 77, 78, 80, 85, 86, 88, 90, 92, 95, 102, 106, 110, 113, 115, 119, 120, 121, 127, 128, 132, 134, 135, 136, 142, 148]


In [ ]:
equal_list = []
for i in s1_list:
    s1path = "/content/drive/MyDrive/SAR/Raw Data/rois1868_summer/s1_" + str(i) + "/"
    s2path = "/content/drive/MyDrive/SAR/Raw Data/rois1868_summer/s2_" + str(i) + "/"
    num_images_s1 = len(os.listdir(s1path))
    num_images_s2 = len(os.listdir(s2path))
    if num_images_s1 == num_images_s2:
        equal_list.append(i)
        print(i)

In [ ]:
print(equal_list)

[0, 10, 18, 19, 20, 24, 102, 106, 110, 113, 115, 119, 120, 121, 127, 128, 132, 134, 135, 136, 142, 148]


In [ ]:
name = 1
for folder in tqdm(equal_list):
    s1path = "/content/drive/MyDrive/SAR/Raw Data/rois1868_summer/s1_" + str(folder) + "/"
    s2path = "/content/drive/MyDrive/SAR/Raw Data/rois1868_summer/s2_" + str(folder) + "/"
    for img_num in range(1, len(os.listdir(s1path))+1):
        s1_img_path = s1path + "ROIs1868_summer_s1_" + str(folder) + "_p" + str(img_num) + ".png" 
        s2_img_path = s2path + "ROIs1868_summer_s2_" + str(folder) + "_p" + str(img_num) + ".png"
        s1_img = cv2.imread(s1_img_path)
        s2_img = cv2.imread(s2_img_path)
        if s1_img is not None and s2_img is not None:
            s1_save_path = "/content/drive/MyDrive/SAR/Raw Data/Summer_Processed/S1/SAR" + str(name) + ".png"
            s2_save_path = "/content/drive/MyDrive/SAR/Raw Data/Summer_Processed/S2/OP" + str(name) + ".png"
            cv2.imwrite(s1_save_path, s1_img)
            cv2.imwrite(s2_save_path, s2_img)
            name+=1
        else:
            continue

100%|██████████| 22/22 [2:21:57<00:00, 387.15s/it]


In [ ]:
gdrives1path = "/content/drive/MyDrive/SAR/Raw Data/Summer_Processed/S1/"
gdrives2path = "/content/drive/MyDrive/SAR/Raw Data/Summer_Processed/S2/"
nums1 = len(os.listdir(gdrives1path))
nums2 = len(os.listdir(gdrives2path))
print(nums1, nums2)

23025 23025


In [ ]:
# from google.colab.patches import cv2_imshow
# s2path_org = "/content/drive/MyDrive/SAR/Raw Data/Summer_Processed/S2/"
# s2path_down = "/content/drive/MyDrive/SAR/Raw Data/Summer_Processed/S2_DownUp/"
# for img in tqdm(range(1, len(os.listdir(s2path_org))+1)):
#     image = cv2.imread("/content/drive/MyDrive/SAR/Raw Data/Summer_Processed/S2/OP" + str(img) + ".png")
#     image_down = cv2.pyrDown(image)
#     image_downup = cv2.pyrUp(image_down)
#     cv2.imwrite("/content/drive/MyDrive/SAR/Raw Data/Summer_Processed/S2_DownUp/Opt" + str(img) + ".png", image_downup)

In [ ]:
image_sar = cv2.imread("/content/drive/MyDrive/SAR/Raw Data/Summer_Processed/S1/SAR1.png")
print(image_sar.shape)

(256, 256, 3)


##U2FusionCode


##Arguments

#Speckle Noise Filtering 

In [ ]:
#convert rgb to greyscale, 
#use speckle denoising
#convert back to rgb

from scipy.ndimage.filters import uniform_filter
from scipy.ndimage.measurements import variance
from pylab import *
import numpy as np

def lee_filter(img, size):
    img_mean = uniform_filter(img, (size, size))
    img_sqr_mean = uniform_filter(img**2, (size, size))
    img_variance = img_sqr_mean - img_mean**2

    overall_variance = variance(img)

    img_weights = img_variance / (img_variance + overall_variance)
    img_output = img_mean + img_weights * (img - img_mean)
    return img_output



# img = np.random.normal(0.5, 0.1, (100,100))



try:
    from .Despeckle_extensions import cy_leeimproved


    class LeeSigma2(pyrat.Worker):
        """
        Lee's improved sigma speckle filter. Fast implementation in Cython.
        J.S. Lee et al.: Improved Sigma Filter for Speckle Filtering of SAR Imagery.
        IEEE Transactions on Geoscience and Remote Sensing, Vol. 47, No.1, pp. 202-213, 2009']
        :author: Andreas Reigber
        """
        gui = {'menu': 'SAR|Speckle filter', 'entry': 'Lee Sigma Improved'}
        para = [
            {'var': 'win', 'value': [9, 9], 'type': 'int', 'range': [3, 999], 'text': 'Window size',
             'subtext': ['range', 'azimuth']},
            {'var': 'looks', 'value': 1.0, 'type': 'float', 'range': [1.0, 99.0], 'text': '# of looks'},
            {'var': 'sigma', 'value': 0.9, 'type': 'float', 'range': [0.0, 1.0], 'text': 'sigma range (0.0-1.0)'},
            {'var': 'perc', 'value': 0.02, 'type': 'float', 'range': [0.0, 1.0],
             'text': 'point target percentile (0.0-1.0)'},
            {'var': 'type', 'value': 'amplitude', 'type': 'list', 'range': ['amplitude', 'intensity'],
             'text': 'SAR data type'}]

        def __init__(self, *args, **kwargs):
            super(LeeSigma2, self).__init__(*args, **kwargs)
            self.name = "IMPROVED SIGMA LEE"
            self.blockprocess = True
            self.blockoverlap = self.win[0] // 2 + 1
            # self.nthreads = 1

        def run(self, *args, **kwargs):
            P = pyrat.tools.ProgressBar('  ' + self.name, 10)
            bounds = opt.fmin(self.optf, [0.5, 2.0], args=(self.looks, self.sigma), disp=False)  # calc sigma bounds
            newsig = self.newsig(bounds[0], bounds[1], sigrng=self.sigma, looks=self.looks)  # calc new stddev
            P.update(0)
            perc = 100.0 - self.perc * 100.0  # point target theshold
            pthreshold = np.mean(self.layer_accumulate(self.estimate_percentile, type=self.type, perc=perc))
            P.update(2)

            layer = self.layer_process(self.leeimproved, bounds=bounds, newsig=newsig,
                                       thres=pthreshold, looks=self.looks, win=self.win, type=self.type)
            P.update(10)
            del P
            pyrat.activate(layer)
            return layer

        @staticmethod
        def leeimproved(array, bounds=(1, 2), thres=10.0, looks=1.0, win=(7, 7), newsig=0.5, type='amplitude',
                        **kwargs):

            if array.ndim == 3:  # polarimetric vector
                if np.iscomplexobj(array) or type == "amplitude":
                    array = np.abs(array) ** 2
                    type = "amplitude"
                else:
                    array = np.abs(array)
                span = np.sum(array, axis=0)
                array = array[np.newaxis, ...]
                type = "amplitude"
            elif array.ndim == 4:  # covariance data
                span = np.abs(np.trace(array, axis1=0, axis2=1))
                type = "intensity"
            else:  # single channel data
                if np.iscomplexobj(array) or type == "amplitude":
                    array = np.abs(array) ** 2
                    type = "amplitude"
                else:
                    array = np.abs(array)
                span = array.copy()
                array = array[np.newaxis, np.newaxis, ...]

            array = cy_leeimproved(span, array, bounds=bounds, thres=thres, looks=looks, win=win, newsig=newsig)
            array[~np.isfinite(array)] = 0.0
            if type == "amplitude":
                array[array < 0] = 0.0
                array = np.sqrt(array)
            return np.squeeze(array)

        def estimate_percentile(self, array, perc=98.0, type='amplitude', **kwargs):
            if array.ndim == 3:  # polarimetric vector
                if np.iscomplexobj(array) or type == "amplitude":
                    span = np.sum(np.abs(array) ** 2, axis=0)
                else:
                    span = np.sum(np.abs(array), axis=0)
            elif array.ndim == 4:  # covariance data
                span = np.abs(np.trace(array, axis1=0, axis2=1))
            else:  # single channel data
                if np.iscomplexobj(array) or type == "amplitude":
                    span = np.abs(array) ** 2
                else:
                    span = np.abs(array)
            return np.percentile(span, perc)

        @staticmethod
        def specklepdf(i, looks=1.0):
            if i < 0.0:
                return 0.0
            else:
                return ((looks ** looks) * (i ** (looks - 1.0))) / sp.special.gamma(looks) * np.exp(-looks * i)

        @staticmethod
        def meanpdf(i, looks=1.0):
            if i < 0.0:
                return 0.0
            else:
                return ((looks ** looks) * (i ** (looks - 1.0))) / sp.special.gamma(looks) * np.exp(-looks * i) * i

        def sigpdf(self, i, looks=1.0):
            return (i - 1.0) ** 2 * self.specklepdf(i, looks=1.0)

        def newsig(self, i1, i2, sigrng=0.9, looks=1.0):
            return 1 / sigrng * sp.integrate.quad(self.sigpdf, i1, i2, args=(looks,))[0]

        def sigmarange(self, i1, i2, looks=1.0):
            return np.clip(sp.integrate.quad(self.specklepdf, i1, i2, args=(looks,))[0], 1e-10, 1.0)

        def intmean(self, i1, i2, looks=1.0):
            return 1.0 / self.sigmarange(i1, i2, looks) * sp.integrate.quad(self.meanpdf, i1, i2, args=(looks,))[0]

        def optf(self, i, looks, sigr):
            return (self.sigmarange(i[0], i[1], looks) - sigr) ** 2 + (self.intmean(i[0], i[1], looks) - 1.0) ** 2


    @pyrat.docstringfrom(LeeSigma2)
    def leesigma2(*args, **kwargs):
        return LeeSigma2(*args, **kwargs).run(**kwargs)

except ImportError:
    logging.info("LeeSigma2 cython module not found. (run build process?)")


In [ ]:
path = "/content/drive/MyDrive/SAR/Testing_Images/SAR/Img"
save_path = "/content/drive/MyDrive/SAR/Testing_Images/SAR_Speckle/Img"
path_dummy = "/content/drive/MyDrive/SAR/Testing_Images/SAR/"

# sar_image = cv2.imread(path + "SAR4.png", cv2.IMREAD_GRAYSCALE)
# speckle_noise_removed = lee_filter(sar_image, 7)
# # cv2_imshow(sar_image)
for i in tqdm(range(1, len(os.listdir(path_dummy))+1)):
    sar_image = cv2.imread(path + str(i) + ".png", cv2.IMREAD_GRAYSCALE)
    speckle_noise_removed = lee_filter(sar_image,2)
    cv2.imwrite(save_path +  str(i) + ".png", speckle_noise_removed)
    # file_name = path_dummy + str(i) + ".png"
    # destination = path_dummy + "SAR" + str(i) + ".png"
    # os.replace(file_name, destination)


# gray = cv2.cvtColor(sar_image, cv2.COLOR_BGR2GRAY)
# print(gray.shape)


# num = len(os.listdir(path))
# for i in tqdm(range(1,num+1)):
    

100%|██████████| 3000/3000 [01:28<00:00, 33.72it/s]


##DataLoader

In [ ]:
class Sen12Train(Dataset):
    def __init__(self, len_path, sar_path, op_path, transform):
        self.sar_path = sar_path
        self.op_path = op_path
        self.len_path = len_path
        
        self.transform=transform
        # self.patch_size = patch_size
    
    def __len__(self):
        # assert (len(os.listdir(self.sar_path))==len(os.listdir(self.op_path)))
        return (len(os.listdir(self.len_path)))

    def __getitem__(self, idx):
        idx += 1
        sar_image = cv2.imread(self.sar_path + str(idx) + ".png")
        sar_image = cv2.cvtColor(sar_image, cv2.COLOR_BGR2YCrCb)
        sar_image = sar_image[:, :, 0:1]
        
              
        op_image = cv2.imread(self.op_path + str(idx) + ".png")    
        op_image = cv2.cvtColor(op_image, cv2.COLOR_BGR2YCrCb)
        op_image = op_image[:, :, 0:1]
        
        sar = self.transform(sar_image)
        optical = self.transform(op_image)
        # sar_image = cv2.resize(sar_image, (224,224))
        # op_image = cv2.resize(op_image, (224,224))

        return (sar, optical)


class Sen12Test(Dataset):
    def __init__(self, sar_path, op_path, transform, patchsize):
        self.sar_path = sar_path
        self.op_path = op_path
        self.transform = transform
        self.patchsize = patchsize

    def __len__(self):
        return (len(os.listdir(self.sar_path)))

    def __getitem__(self, idx):
        idx += 1
        sar_image = cv2.imread(self.sar_path + "Img" + str(idx) + ".png")  
        sar_image_1 = cv2.cvtColor(sar_image, cv2.COLOR_BGR2YCrCb) 
        
        # sar_image = sar_image[:, :, 0:1]
        op_image = cv2.imread(self.op_path + "Img" + str(idx) + ".png") 
        op_image_1 = cv2.cvtColor(op_image, cv2.COLOR_BGR2YCrCb) 
        # op_image = op_image[:, :, 0:1]
        if self.transform:
            sar = self.transform(sar_image_1)
            optical = self.transform(op_image_1)
        img_stack = torch.stack((sar, optical), 0)
        return img_stack


    
# class MEFdataset(data.Dataset):
#     def __init__(self, transform, patch_size):
#         super(MEFdataset, self).__init__()
#         self.dir_prefix = args.dir_train
#         self.over = os.listdir(self.dir_prefix + 'over/')
#         self.under = os.listdir(self.dir_prefix + 'under/')

#         self.patch_size = args.patch_size
#         self.transform = transform

#     def __len__(self):
#         assert len(self.over) == len(self.under)
#         return len(self.over)

#     def __getitem__(self, idx):
#         over = cv2.imread(self.dir_prefix + 'over/' + self.over[idx])
#         over = cv2.cvtColor(over, cv2.COLOR_BGR2YCrCb)
#         over = over[:, :, 0:1]
#         under = cv2.imread(self.dir_prefix + 'under/' + self.under[idx])
#         under = cv2.cvtColor(under, cv2.COLOR_BGR2YCrCb)
#         under = under[:, :, 0:1]

#         over_p, under_p = self.get_patch(over, under)
#         if self.transform:
#             over_p = self.transform(over_p)
#             under_p = self.transform(under_p)

#         return over_p, under_p

#     def get_patch(self, over, under):
#         h, w = over.shape[:2]
#         stride = self.patch_size

#         x = random.randint(0, w - stride)
#         y = random.randint(0, h - stride)

#         over = over[y:y + stride, x:x + stride, :]
#         under = under[y:y + stride, x:x + stride, :]

#         return over, under


# class TestData(data.Dataset):
#     def __init__(self, transform):
#         super(TestData, self).__init__()
#         self.transform = transform
#         self.dir_prefix = args.dir_test
#         self.over_dir = os.listdir(self.dir_prefix + 'over/')
#         self.under_dir = os.listdir(self.dir_prefix + 'under/')

#     def __getitem__(self, idx):
#         over = cv2.imread(self.dir_prefix + 'over/' + self.over_dir[idx])
#         under = cv2.imread(self.dir_prefix + 'under/' + self.under_dir[idx])
#         over_img = cv2.cvtColor(over, cv2.COLOR_BGR2YCrCb)
#         under_img = cv2.cvtColor(under, cv2.COLOR_BGR2YCrCb)

#         if self.transform:
#             over_img = self.transform(over_img)
#             under_img = self.transform(under_img)

#         img_stack = torch.stack((over_img, under_img), 0)
#         return img_stack

#     def __len__(self):
#         assert len(self.over_dir) == len(self.under_dir)
#         return len(self.over_dir)

##Utils

In [ ]:
# tool functions
def get_img_seq(img_seq_dir):
    img_seq = []
    for root, _, fnames in sorted(os.walk(img_seq_dir)):
        for fname in sorted(fnames):
            if any(fname.endswith(ext) for ext in args.ext):
                img_name = os.path.join(root, fname)
                img_seq.append(cv2.imread(img_name))
    return img_seq


def features_grad(features):
    kernel = [[1 / 8, 1 / 8, 1 / 8], [1 / 8, -1, 1 / 8], [1 / 8, 1 / 8, 1 / 8]]
    kernel = torch.FloatTensor(kernel).unsqueeze(0).unsqueeze(0)
    kernel = kernel.cuda()
    _, c, _, _ = features.shape
    c = int(c)
    for i in range(c):
        feat_grad = F.conv2d(features[:, i:i + 1, :, :], kernel, stride=1, padding=1)
        if i == 0:
            feat_grads = feat_grad
        else:
            feat_grads = torch.cat((feat_grads, feat_grad), dim=1)
    return feat_grads


# network functions
def pad(pad_type, padding):
    pad_type = pad_type.lower()
    if padding == 0:
        return None


def get_valid_padding(kernel_size, dilation):
    kernel_size = kernel_size + (kernel_size - 1) * (dilation - 1)
    padding = (kernel_size - 1) // 2
    return padding


def activation(act_type, slope=0.2, n_prelu=1):
    act_type = act_type.lower()
    if act_type == 'prelu':
        layer = nn.PReLU(num_parameters=n_prelu, init=slope)
    elif act_type == 'lrelu':
        layer = nn.LeakyReLU(negative_slope=slope, inplace=True)
    else:
        raise NotImplementedError('[ERROR] Activation layer [%s] is not implemented!' % act_type)
    return layer


def norm(n_feature, norm_type='bn'):
    norm_type = norm_type.lower()
    if norm_type == 'bn':
        layer = nn.BatchNorm2d(n_feature)
    else:
        raise NotImplementedError('[ERROR] %s.sequential() does not support OrderedDict' % norm_type)
    return layer


def sequential(*args):
    if len(args) == 1:
        if isinstance(args[0], OrderedDict):
            raise NotImplementedError('[ERROR] %s.sequential() does not support OrderedDict' % sys.modules[__name__])
        else:
            return args[0]
    modules = []
    for module in args:
        if isinstance(module, nn.Sequential):
            for submodule in module:
                modules.append(submodule)
        elif isinstance(module, nn.Module):
            modules.append(module)
    return nn.Sequential(*modules)


def ConvBlock(in_channels, out_channels, kernel_size, stride=1, dilation=1, bias=True, valid_padding=True, padding=0,
              act_type='prelu', norm_type='bn', pad_type='zero'):
    if valid_padding:
        padding = get_valid_padding(kernel_size, dilation)
    else:
        pass
    p = pad(pad_type, padding) if pad_type and pad_type != 'zero' else None
    conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=padding, dilation=dilation,
                     bias=bias)

    act = activation(act_type) if act_type else None
    n = norm(out_channels, norm_type) if norm_type else None
    return sequential(p, conv, n, act)

##Model - DenseLayer

In [ ]:
class DenseLayer(nn.Module):
    def __init__(self, num_channels, growth):
        super(DenseLayer, self).__init__()
        self.conv = ConvBlock(num_channels, growth, kernel_size=3, act_type='lrelu', norm_type=None)

    def forward(self, x):
        out = self.conv(x)
        out = torch.cat((x, out), 1)
        return out


class DenseNet(nn.Module):
    def __init__(self, in_channels, num_features, growth, num_layers):
        super(DenseNet, self).__init__()
        self.num_channels = 2 * in_channels
        self.num_features = num_features
        self.growth = growth
        modules = []
        self.conv_1 = ConvBlock(self.num_channels, self.num_features, kernel_size=3, act_type='lrelu', norm_type=None)
        for i in range(num_layers):
            modules.append(DenseLayer(self.num_features, self.growth))
            self.num_features += self.growth
        self.dense_layers = nn.Sequential(*modules)
        self.sub = nn.Sequential(ConvBlock(self.num_features, 128, kernel_size=3, act_type='lrelu', norm_type=None),
                                 ConvBlock(128, 64, kernel_size=3, act_type='lrelu', norm_type=None),
                                 ConvBlock(64, 32, kernel_size=3, act_type='lrelu', norm_type=None),
                                 nn.Conv2d(32, in_channels, kernel_size=3, stride=1, padding=1),
                                 nn.Tanh())

    def forward(self, x_over, x_under):
        x = torch.cat((x_over, x_under), dim=1)
        x = self.conv_1(x)
        x = self.dense_layers(x)
        x = self.sub(x)
        return x

##VGG

In [ ]:
_all__ = [
    'VGG', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn',
    'vgg19_bn', 'vgg19',
]


model_urls = {
    'vgg11': 'https://download.pytorch.org/models/vgg11-bbd30ac9.pth',
    'vgg13': 'https://download.pytorch.org/models/vgg13-c768596a.pth',
    'vgg16': 'https://download.pytorch.org/models/vgg16-397923af.pth',
    'vgg19': 'https://download.pytorch.org/models/vgg19-dcbb9e9d.pth',
}


class VGG(nn.Module):

    def __init__(self, features, num_classes=1000):
        super(VGG, self).__init__()
        self.features = features
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )
        self._initialize_weights()

    def forward(self, x):
        x_1 = self.features[3](x)
        x_2 = self.features[8](x)
        x_3 = self.features[15](x)
        x_4 = self.features[22](x)
        x_5 = self.features[29](x)
        # x = self.features(x)
        # x = x.view(x.size(0), -1)
        # x = self.classifier(x)
        return x_1, x_2, x_3, x_4, x_5

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                n = m.weight.size(1)
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()


def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 3
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)


cfg = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}


def vgg11(pretrained=False, model_root=None, **kwargs):
    """VGG 11-layer model (configuration "A")"""
    model = VGG(make_layers(cfg['A']), **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['vgg11'], model_root))
    return model


def vgg11_bn(**kwargs):
    """VGG 11-layer model (configuration "A") with batch normalization"""
    kwargs.pop('model_root', None)
    return VGG(make_layers(cfg['A'], batch_norm=True), **kwargs)


def vgg13(pretrained=False, model_root=None, **kwargs):
    """VGG 13-layer model (configuration "B")"""
    model = VGG(make_layers(cfg['B']), **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['vgg13'], model_root))
    return model


def vgg13_bn(**kwargs):
    """VGG 13-layer model (configuration "B") with batch normalization"""
    kwargs.pop('model_root', None)
    return VGG(make_layers(cfg['B'], batch_norm=True), **kwargs)


def vgg16(pretrained=False, model_root=None, **kwargs):
    """VGG 16-layer model (configuration "D")"""
    model = VGG(make_layers(cfg['D']), **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['vgg16'], model_root))
    return model


def vgg16_bn(**kwargs):
    """VGG 16-layer model (configuration "D") with batch normalization"""
    kwargs.pop('model_root', None)
    return VGG(make_layers(cfg['D'], batch_norm=True), **kwargs)


def vgg19(pretrained=False, model_root=None, **kwargs):
    """VGG 19-layer model (configuration "E")"""
    model = VGG(make_layers(cfg['E']), **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['vgg19'], model_root))
    return model


def vgg19_bn(**kwargs):
    """VGG 19-layer model (configuration 'E') with batch normalization"""
    kwargs.pop('model_root', None)
    return VGG(make_layers(cfg['E'], batch_norm=True), **kwargs)


# if __name__ == '__main__':
#     vgg16_net = vgg16()
#     print(vgg16_net)

In [ ]:
vgg16_net = vgg16()
print(vgg16_net)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

##Train Test

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
print(device)

cuda


In [ ]:
class Train(object):
    def __init__(self, epochs, batch_size, model_path, 
                 model_name, sar_path, op_path, len_path, 
                 patchsize, learning_rate, validation, 
                 in_channels, growth, num_features, num_layers, c):
        self.num_epochs = epochs
        self.lr = learning_rate
        self.model_path = model_path
        self.model_name = model_name
        self.sar_path = sar_path
        self.op_path = op_path
        self.batch_size = batch_size
        self.in_channels = in_channels
        self.growth = growth
        self.num_features = num_features
        self.num_layers = num_layers
        self.patchsize = patchsize
        self.c = c
        self.len_path = len_path

        self.transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=[0.5], std=[0.5])])
        self.train_set = Sen12Train(self.len_path, self.sar_path, self.op_path, self.transform)
        
        self.train_loader = DataLoader(self.train_set, batch_size = self.batch_size,
                                            shuffle=True, num_workers=4)

        self.model = DenseNet(self.in_channels, self.num_features, self.growth, self.num_layers).to(device)
        self.feature_model = vgg16().to(device)
        self.feature_model.load_state_dict(torch.load('/content/drive/MyDrive/SAR/VGG/vgg16.pth'))
        self.optimizer = Adam(self.model.parameters(), lr=self.lr)
        self.scheduler = lr_scheduler.ExponentialLR(self.optimizer, gamma=0.9)

        self.loss_mse = nn.MSELoss(reduction='mean').to(device)
        self.train_loss_1 = []
        self.train_loss_2 = []
        if validation:
            self.val_list = []
            self.best_psnr = 0

    def train(self):
        # seed = args.seed
        # random.seed(seed)
        # torch.manual_seed(seed)
        # writer = SummaryWriter(log_dir=args.log_dir, filename_suffix='train_loss')

        if os.path.exists(self.model_path + self.model_name + ".pth"):
            print('===> Loading pre-trained model......')
            state = torch.load(self.model_path + self.model_name + ".pth")
            self.model.load_state_dict(state['model'])
            self.train_loss_1 = state['train_loss_1']
            self.train_loss_2 = state['train_loss_2']
            # self.lr = state['lr']

        for ep in range(self.num_epochs):
            ep_loss_1 = []
            ep_loss_2 = []
            for batch, (sar, optical) in tqdm(enumerate(self.train_loader)):
                # sar = (sar + 1) / 2
                sar = sar.to(device)
                # optical = (optical + 1) / 2
                optical = optical.to(device)

                with torch.no_grad():
                    feat_1 = torch.cat((sar, sar, sar), dim=1)
                    feat_1 = self.feature_model(feat_1)
                    feat_2 = torch.cat((optical, optical, optical), dim=1)
                    feat_2 = self.feature_model(feat_2)

                    for i in range(len(feat_1)):
                        m1 = torch.mean(features_grad(feat_1[i]).pow(2), dim=[1, 2, 3])
                        m2 = torch.mean(features_grad(feat_2[i]).pow(2), dim=[1, 2, 3])
                        if i == 0:
                            w1 = torch.unsqueeze(m1, dim=-1)
                            w2 = torch.unsqueeze(m2, dim=-1)
                        else:
                            w1 = torch.cat((w1, torch.unsqueeze(m1, dim=-1)), dim=-1)
                            w2 = torch.cat((w2, torch.unsqueeze(m2, dim=-1)), dim=-1)
                    weight_1 = torch.mean(w1, dim=-1) / self.c
                    weight_2 = torch.mean(w2, dim=-1) / self.c
                    weight_list = torch.cat((weight_1.unsqueeze(-1), weight_2.unsqueeze(-1)), -1)
                    weight_list = F.softmax(weight_list, dim=-1)

                # sar = sar.to(device)
                # optical = optical.to(device)

                self.optimizer.zero_grad()
                # torch.cuda.synchronize()
                start_time = time.time()
                fused_img = self.model(sar, optical)
                # torch.cuda.synchronize()
                end_time = time.time()
                # fused_img = (fused_img + 1) / 2

                # sar = (sar + 1) / 2
                # optical = (optical + 1) / 2

                loss_1 = weight_list[:, 0] * (1 - ssim(fused_img, sar, nonnegative_ssim=True)) \
                         + weight_list[:, 1] * (1 - ssim(fused_img, optical, nonnegative_ssim=True))
                loss_1 = torch.mean(loss_1)

                loss_2 = weight_list[:, 0] * self.loss_mse(fused_img, sar) \
                         + weight_list[:, 1] * self.loss_mse(fused_img, optical)
                loss_2 = torch.mean(loss_2)

                loss = loss_1 + 20 * loss_2
                ep_loss_1.append(loss_1.item())
                ep_loss_2.append(loss_2.item())
                loss.backward()
                self.optimizer.step()
                del sar
                del optical

                if batch % 50 == 0 and batch != 0:
                    print('Epoch:{}\tcur/all:{}/{}\tLoss_1:{:.4f}\tLoss_2:{:.4f}\t'
                          'Time:{:.2f}s'.format(ep + 1, batch,
                                                len(self.train_loader),
                                                loss_1.item(),
                                                loss_2.item(),
                                                end_time - start_time))
                    state = {
                    'epoch' : ep, 
                    'model': self.model.state_dict(),
                    'train_loss_1': loss_1.item(),
                    'train_loss_2': loss_2.item(),
                    'lr': self.optimizer.param_groups[0]['lr']
                        }
                    torch.save(state, self.model_path + self.model_name + '.pth')

            self.scheduler.step()
            # self.train_loss_1.append(np.mean(ep_loss_1))
            # self.train_loss_2.append(np.mean(ep_loss_2))

            
            if ep % 5 == 0:
                torch.save(state, self.model_path + self.model_name + str(ep) + '.pth')
            # matplotlib.use('Agg')
            # fig1 = plt.figure()
            # plot_loss_list_1 = self.train_loss_1
            # plt.plot(plot_loss_list_1)
            # plt.savefig('train_loss_curve_1.png')
            # fig2 = plt.figure()
            # plot_loss_list_2 = self.train_loss_2
            # plt.plot(plot_loss_list_2)
            # plt.savefig('train_loss_curve_2.png')

            # writer.add_scalar('ssim_loss', np.mean(ep_loss_1), ep)
            # writer.add_scalar('mse_loss', np.mean(ep_loss_2), ep)

            # if train_test:
            #     t = Test(ep)
            #     t.test()
        print('===> Finished Training!')


class Test(object):
    def __init__(self, batch_size, model_path, model_name, 
                 patchsize, sar_path, op_path, 
                 in_channels, growth, num_features, num_layers, eps, c, save):
        
        self.transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                                                                         std=[0.5, 0.5, 0.5])])
        self.model_path = model_path
        self.model_name = model_name
        self.sar_path = sar_path
        self.op_path = op_path
        self.batch_size = batch_size
        self.in_channels = in_channels
        self.growth = growth
        self.num_features = num_features
        self.num_layers = num_layers
        self.patchsize = patchsize
        self.EPS = eps
        self.c = c
        self.save_dir = save_dir

        self.test_set = Sen12Test(self.sar_path, self.op_path, self.transform, self.patchsize)
        self.test_loader = data.DataLoader(self.test_set, batch_size=self.batch_size, shuffle=False,
                                           num_workers=0, pin_memory=False)
        self.model = DenseNet(self.in_channels, self.num_features, self.growth, self.num_layers).to(device)
        self.state = torch.load(self.model_path + self.model_name)
        self.model.load_state_dict(self.state['model'])

    def test(self):
        self.model.eval()
        with torch.no_grad():
            for batch, images in tqdm(enumerate(self.test_loader)):
                print('Processing picture No.{}'.format(batch + 1))
                # print("Starting", imgs.shape)
                imgs = torch.squeeze(images, dim=0)
                # print("After Squeeze", imgs.shape)
                img1_y = imgs[0:1, 0:1, :, :].cuda()
                # print("img1_y", img1_y.shape)
                img2_y = imgs[1:2, 0:1, :, :].cuda()
                # print("img2_y", img2_y.shape)

                img_cr = imgs[:, 1:2, :, :].cuda()
                # print("img_cb", img_cr.shape)
                img_cb = imgs[:, 2:3, :, :].cuda()
                # print("img_cb", img_cb.shape)
                w_cr = (torch.abs(img_cr) + self.EPS) / torch.sum(torch.abs(img_cr) + self.EPS, dim=0)
                w_cb = (torch.abs(img_cb) + self.EPS) / torch.sum(torch.abs(img_cb) + self.EPS, dim=0)
                fused_img_cr = torch.sum(w_cr * img_cr, dim=0, keepdim=True).clamp(-1, 1)
                fused_img_cb = torch.sum(w_cb * img_cb, dim=0, keepdim=True).clamp(-1, 1)

                fused_img_y = self.model(img1_y, img2_y)
                # print("Fused_Image_y", fused_img_y.shape)
                fused_img = torch.cat((fused_img_y, fused_img_cr, fused_img_cb), dim=1)
                # print("Fused Image after concatenating cb and cr bands", fused_img.shape)
                fused_img = (fused_img + 1) * 127.5
                # print("Fused Image after normalizing", fused_img.shape)
                fused_img = fused_img.squeeze(0)
                # print("Fused Image after squeezing", fused_img.shape)
                fused_img = fused_img.cpu().numpy()
                fused_img = np.transpose(fused_img, (1, 2, 0))
                # print("Fused Image after permute", fused_img.shape)
                fused_img = fused_img.astype(np.uint8)
                fused_img = cv2.cvtColor(fused_img, cv2.COLOR_YCrCb2BGR)
                # print("Fused Image after convert to BGR", fused_img.shape)


                # if self.ep:
                #     save_path = args.save_dir + str(self.ep) + '_epoch/'
                # else:
                #     save_path = args.save_dir
                cv2.imwrite((self.save_dir + "Fused/Img" + str(batch + 1) + ".png"), fused_img)

            print('Finished testing!')

##Model Training Main

In [ ]:
EPS = 1e-8
c_pass = 3500

##defining arguments 

val = False
test_only = False
train_test = True

num_epochs = 5
lr = 1e-5
test_epochs = 0
log_dir = "./train_log"
train_dir = ""
val_dir = ""
test_dir = ""
path_model = "/content/drive/MyDrive/SAR/TrainedModels/"
name_model = "U2Fusion_WithoutSpeckle_WithNorm"
sar_p = "/content/drive/MyDrive/SAR/Raw Data/Summer_Processed/S1_Speckle/SAR"
op_p = "/content/drive/MyDrive/SAR/Raw Data/Summer_Processed/S2/OP"

len_p = "/content/drive/MyDrive/SAR/Raw Data/Summer_Processed/S1_Speckle/"
extension = ".png"

btsize = 24
patch = 64
in_ch = 1
out_channels = 3
num_feat = 44
growth_rate = 44
num_lay= 5
activation_type = "prelu"
eval = True


In [ ]:
t = Train(epochs=num_epochs, batch_size = btsize, 
          model_path=path_model, model_name = name_model, 
          sar_path = sar_p, op_path = op_p, len_path = len_p, patchsize = patch, 
          learning_rate = lr, validation = val, in_channels = in_ch, 
          growth = growth_rate, num_features = num_feat, num_layers = num_lay, c = c_pass)
t.train()


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
50it [01:33,  1.33s/it]

Epoch:1	cur/all:50/835	Loss_1:0.0252	Loss_2:0.1997	Time:0.00s


101it [02:41,  1.80s/it]

Epoch:1	cur/all:100/835	Loss_1:0.0139	Loss_2:0.1173	Time:0.00s


151it [03:59,  1.51s/it]

...

Epoch:5	cur/all:750/835	Loss_1:0.0184	Loss_2:0.1408	Time:0.00s


801it [13:32,  1.18s/it]

Epoch:5	cur/all:800/835	Loss_1:0.0188	Loss_2:0.1410	Time:0.00s


835it [14:04,  1.01s/it]

===> Finished Training!


In [ ]:
test_batch_size = 1
test_path_model = "/content/drive/MyDrive/SAR/TrainedModels/"
test_name_model = "U2Fusion_WithoutSpeckle_WithNorm.pth"
save_dir = "/content/drive/MyDrive/SAR/Testing_Images/U2Fusion_WithoutSpeckle/"

sar_path_test = "/content/drive/MyDrive/SAR/Testing_Images/SAR_Speckle/"
op_path_test = "/content/drive/MyDrive/SAR/Testing_Images/Optical/"

t = Test(batch_size = test_batch_size, 
          model_path=test_path_model, model_name = test_name_model, 
          patchsize = patch, sar_path = sar_path_test, op_path = op_path_test, 
          in_channels = in_ch, growth = growth_rate, num_features = num_feat, num_layers = num_lay, 
          eps = EPS, c = c_pass, save = save_dir)
t.test()

0it [00:00, ?it/s]

Processing picture No.1


2it [00:01,  1.92it/s]

Processing picture No.2


3it [00:01,  2.42it/s]

...

Processing picture No.2998


2999it [14:08,  3.45it/s]

Processing picture No.2999


3000it [14:08,  3.53it/s]

Processing picture No.3000
Finished testing!


In [ ]:
a = len(os.listdir(sar_p))
b = len(os.listdir(op_p))

print(a,b)

23025 23025


In [ ]:
sar_image = np.array(cv2.imread("/content/drive/MyDrive/SAR/Raw Data/Summer_Processed/S1/" + "SAR1.png"))
print(len(os.listdir("/content/drive/MyDrive/SAR/Raw Data/Summer_Processed/S1/")))
print(sar_image.shape)

KeyboardInterrupt: ignored

##Creating Memmaps


In [ ]:
path = "/content/drive/MyDrive/SAR/Raw Data/Summer_Processed/"
sar_array = np.zeros((5000, 256, 256, 3))
# op_array = np.zeros((5000, 256, 256, 3))
for i in tqdm(range(1,5001)):
    sar_image = np.array(cv2.imread(path + "S1/SAR" + str(i) + ".png"))
    # optical_image = np.array(cv2.imread(path + "S2/OP" + str(i) + ".png"))
    sar_array[i-1] = sar_image
    # op_array[i-1] = optical_image
np.save("/content/drive/MyDrive/SAR/Raw Data/Summer_Processed/SARTrainArray1.npy", sar_array)
# np.save("/content/drive/MyDrive/SAR/Raw Data/Summer_Processed/OpticalTrainArray1.npy", op_array)

100%|██████████| 5000/5000 [1:17:07<00:00,  1.08it/s]


In [ ]:
# filename1 = "/content/drive/MyDrive/SAR/Raw Data/Summer_Processed/SARTrainArray2_Mem.mem"
filename2 = "/content/drive/MyDrive/SAR/Raw Data/Summer_Processed/OpticalTrainArray1_Mem.mem"
# fp = np.memmap(filename1, dtype='float32', mode='w+', shape=(3000,256, 256, 3))
newfp = np.memmap(filename2, dtype='float32', mode='w+', shape=(5000,256, 256, 3))
# fp[:] = sar_array[:]
newfp[:] = op_array[:]
newfp.flush()

In [ ]:
del op_array

In [ ]:
sar_array = np.load("/content/drive/MyDrive/SAR/Raw Data/Summer_Processed/TrainArray1.npy")
filename1 = "/content/drive/MyDrive/SAR/Raw Data/Summer_Processed/SARTrainArray2_Mem.mem"
fp = np.memmap(filename1, dtype='float32', mode='w+', shape=(3000,256, 256, 3))
fp[:] = sar_array[:]
print(fp)
fp.flush()

FileNotFoundError: ignored

##CustomImages

In [ ]:
import tifffile

In [ ]:
img = np.array(tifffile.imread('/content/drive/MyDrive/SAR_Optical/SAR_VV_Sigma0_VH.tif'))
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
_,thresh = cv2.threshold(gray,1,255,cv2.THRESH_BINARY)

contours,hierarchy = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnt = contours[0]
x,y,w,h = cv2.boundingRect(cnt)

crop = img[y:y+h,x:x+w]
cv2.imwrite('/content/drive/MyDrive/SAR_Optical/SAR_Black_Removed.png',crop)

True

In [ ]:
sar_img = np.array(cv2.imread('/content/drive/MyDrive/SAR_Optical/SAR_Black_Removed.png'))
optical_img = np.array(tifffile.imread('/content/drive/MyDrive/SAR_Optical/clipped.tif'))
print(sar_img.shape)
print(optical_img.shape)

(818, 819, 3)
(1829, 1803, 4)


In [ ]:
optical_img = cv2.resize(optical)

##Debugging

In [ ]:
sar_path = "/content/drive/MyDrive/SAR/Raw Data/Summer_Processed/S1/"
op_path = "/content/drive/MyDrive/SAR/Raw Data/Summer_Processed/S2/"

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=[0.5], std=[0.5])])
train_set = Sen12Train(sar_path, op_path, transform)

train_loader = DataLoader(train_set, batch_size = 1, shuffle=True, num_workers=0)

start_time = time.time()
for batch, (sar, optical) in tqdm(enumerate(train_loader)): 
    continue
end_time = time.time()
time_taken = end_time - start_time           

8it [00:00, 17.34it/s]

14
15
2
12
13
19
21
7
16
9
11


11it [00:00, 17.10it/s]


0


error: ignored

In [ ]:
names = ['MSE', 'PSNR', 'SCC', 'SAM', 'SSIM']
global metrics 
metrics = [0] * len(names)

def FullRef_Metrics(predicted_image, groundtruth_image):
  
    results = []
    t = 0
    for i in tqdm(range(0,total)):
        predictedimage = predicted[i]
        gtimage = groundtruth[i]
        s = sam(gtimage, predictedimage)
        if math.isnan(s) == True:
            continue
        else:
            mse += mse(gtimage, predictedimage)
            # metrics[1] += rmse(gtimage, predictedimage)
            psnr += psnr(gtimage, predictedimage, MAX = 2047) # The value of MAX changes according to the bit level of the image
            # metrics[3] += uqi(gtimage, predictedimage, ws = 2)
            # metrics[4] += scc(gtimage, predictedimage)
            # metrics[5] += s
            ssm, cs = ssim(gtimage, predictedimage, ws = 2, MAX = 2047)
            metrics[6] += ssm
        return s, mse, psnr, ssim 
    
    
    for i in metrics:
      results.append(i/t)
    mets = {}
    for i in range(0, len(names)):
        mets[names[i]] = results[i]
    l = list(mets.items())
    dt = pd.DataFrame(l,columns = ['Metrics','Values'])
    return dt